In [ ]:
import os 
import sys
import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
plt.rcParams["font.size"] = 15
#import seaborn as sns
import json
from tqdm import tqdm
import pandas as pd
pd.set_option("display.max_rows", 101)
import glob
from collections import Counter
from PIL import Image

input_dir = "./advanced_ML/data/"

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from keras import backend as K
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

In [ ]:
train_df = pd.read_csv(input_dir + "train.csv")

In [ ]:
train_df

In [ ]:

def classid2label(class_id):
    category, *attribute = class_id.split("_")
    return category, attribute




def json2df(data):
    df = pd.DataFrame()
    for index, el in enumerate(data):
        for key, val in el.items():
            df.loc[index, key] = val
    return df



input_dir = "advanced_ML/data/"
train_df = pd.read_csv(input_dir + "train.csv")
train_df.head()



with open(input_dir + "label_descriptions.json") as f:
    label_description = json.load(f)
    
print("this dataset info")
print(json.dumps(label_description["info"], indent=2))




category_df = json2df(label_description["categories"])
category_df["id"] = category_df["id"].astype(int)
category_df["level"] = category_df["level"].astype(int)
attribute_df = json2df(label_description["attributes"])
attribute_df["id"] = attribute_df["id"].astype(int)
attribute_df["level"] = attribute_df["level"].astype(int)


# In[ ]:


print("Category Labels")
#category_df


# In[ ]:


print("Attribute Labels")
#attribute_df


# In[ ]:


#sample_rle_encoding=train_df['EncodedPixels'][0]


# In[ ]:


counter_category = Counter()
counter_attribute = Counter()
for class_id in train_df["ClassId"]:
    category, attribute = classid2label(class_id)
    counter_category.update([category])
    counter_attribute.update(attribute)


# In[ ]:


category_name_dict = {}
for i in label_description["categories"]:
    category_name_dict[str(i["id"])] = i["name"]
attribute_name_dict = {}
for i in label_description["attributes"]:
    attribute_name_dict[str(i["id"])] = i["name"]


# In[ ]:


ROOT_DIR="fashion/mrcnn"
DATA_DIR="advanced_ML/data"


In [ ]:
sys.path.append(os.path.join(ROOT_DIR, 'Mask_RCNN'))

In [ ]:
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [ ]:
class FashionDataset(utils.Dataset):
    def load_fashion(self,image_ids=None,num_data=None):
        '''
        add_class --> register 46 classes self.add_class('fashion',i,name)
        image_ids --> unique index of images(name?)
        self.add_image('fashion',image_ids,width,height,annotations)
        width,height --> shape[:2] or extract form dataframe
        annotations --> all collections of annotations for each image
        Todo:
        There are some rows that have height and weight as nan value
        validation option is necessary for training
        '''
        for i,row in category_df.iterrows():
            self.add_class('fashion',i,row['name'])
        
        if image_ids is None:
            image_ids = list(set(train_df['ImageId']))
        
        if num_data is not None:
            random.seed(42)
            random.shuffle(image_ids)
            image_ids=image_ids[:num_data]
            
            
        for i in image_ids:
            Width = train_df[train_df['ImageId']==i]['Width'].reset_index(drop=True)[0]
            Height = train_df[train_df['ImageId']==i]['Height'].reset_index(drop=True)[0]
            self.add_image('fashion',
                           image_id=i,
                           path=DATA_DIR+'/train/'+i,
                           width=Width,
                           height=Height,
                           annotations=train_df[train_df['ImageId']==i])
        
        
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

    def load_image(self, image_id):
        info = self.image_info[image_id]
        ImagePath = info['path']
        image = np.asarray(Image.open(ImagePath).convert("RGB"))
        return image

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        annotations = info['annotations']
        width=info['width']
        height=info['height']
        
        instance_masks = []
        class_ids = []
        
        for i,annotation in annotations.iterrows():
            class_id=annotation['ClassId']
            class_id=class_id.split('_')[0]
            class_ids.append(class_id)
            rle = annotation['EncodedPixels']
            instance_masks.append(self.rle_to_mask(rle,width,height))
            
        if class_ids:
            mask = np.stack(instance_masks, axis=2).astype(np.bool)
            class_ids = np.array(class_ids, dtype=np.int32)+1
            return mask, class_ids
        else:
            # Call super class to return an empty mask
            return super(FashionDataset, self).load_mask(image_id)
        
        
    def rle_to_mask(self,rle,width,height):
        mask = np.zeros(width*height,dtype=np.int8)
        pixels_list = list(map(int,rle.split(" ")))
        for i in range(0,len(pixels_list),2):
            start_pixel = pixels_list[i]-1
            num_pixel = pixels_list[i+1]-1
            mask[start_pixel:start_pixel+num_pixel] = 1
        
        mask = mask.reshape((height,width),order='F')
        return mask
    
    def load_attributes(self, image_id):
        info = self.image_info[image_id]
        annotations = info['annotations']
        
        return annotations['ClassId'].apply(lambda x: x.split('_'))

In [ ]:

image_ids_list = list(set(train_df['ImageId']))

random.seed(42)
random.shuffle(image_ids_list)

val_split = 0.1
split = int((1-val_split)*len(image_ids_list))

#train_ids = image_ids_list[:]
val_ids = image_ids_list[split:]
#train_ids = train_ids[:3000]
val_ids = val_ids[:100]


In [ ]:
#fashion_dataset_train = FashionDataset()
#fashion_dataset_train.load_fashion(train_ids)
fashion_dataset_val = FashionDataset()
fashion_dataset_val.load_fashion(val_ids)

In [ ]:
class FashionConfig(Config):

    NAME = "fashion"

    IMAGES_PER_GPU = 1

    # Uncomment to train on 8 GPUs (default is 1)
    GPU_COUNT = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 46
    STEPS_PER_EPOCH=100


class InferenceConfig(FashionConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
    
inference_config = InferenceConfig()
inference_config.display()

In [ ]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", config=inference_config,model_dir=ROOT_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
#model_path = model.find_last()
# print("Loading weights from ", model_path)
#model_path='fashion/mrcnn/fashion20190608T2206/mask_rcnn_fashion_0044.h5'
#model_path='fashion/mrcnn/pre-trained/mask_rcnn_fashion_0015.h5'
model_path='fashion/mrcnn/fashion20190613T0413/mask_rcnn_fashion_0030.h5'

In [ ]:
model.load_weights(model_path, by_name=True)

In [ ]:
fashion_dataset_val.prepare()
#fashion_dataset_train.image_ids

In [ ]:
'''
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(fashion_dataset_val.image_ids, 100)
Apss=[]
for iou in tqdm(np.arange(0.5,1,0.05)):
    APs = []
    for image_id in image_ids:
        # Load image and ground truth data
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(fashion_dataset_val, inference_config,
                                   image_id, use_mini_mask=False)
        molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
        # Run object detection
        results = model.detect([image], verbose=0)
        r = results[0]
        # Compute AP
        AP, precisions, recalls, overlaps =\
            utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                             r["rois"], r["class_ids"], r["scores"], r['masks'],iou_threshold=iou)
        APs.append(AP)
    
    Apss.append(np.mean(APs))
# plt.plot(np.arange(0.5,1,0.05),Apss)
'''

In [ ]:
'''
plt.plot(np.arange(0.5,1,0.05),Apss)
plt.ylabel("mAP")
plt.xlabel("Threshold")
'''

In [ ]:
img_ids=np.random.choice(fashion_dataset_train.image_ids,10)
img = fashion_dataset_train.load_image(img_ids[6])
plt.imshow(img)

In [ ]:
!ls fashion/realdata

In [ ]:
img=np.array(Image.open('fashion/realdata/chams.jpg'))

In [ ]:
plt.imshow(img)

In [ ]:
img=np.rot90(img,3)

In [ ]:
plt.imshow(img)

In [ ]:

#i=6

#visualize.display_instances(img, np.expand_dims(r['rois'][i],0), np.expand_dims(r['masks'][:,:,i],-1), np.array([r['class_ids'][i]]), 
#                            class_names)

In [ ]:
'''
bboxs=r['rois']
raw_masks=r['raw_masks']
molded_masks=r['masks']
img_seg=img[bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
mask_seg=raw_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
molded_mask=molded_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
'''

In [ ]:
'''
probs=r['raw_masks']
bbi=r['rois'][5]
bbimg=img[bbi[0]:bbi[2],bbi[1]:bbi[3]]
bbprobs=probs[:,:,5][bbi[0]:bbi[2],bbi[1]:bbi[3]]
bbprobs.sum()
'''

In [ ]:
'''
a=np.expand_dims(probs[:,:,0],0)
a=np.append(1-a,a,axis=0)
np.log((a[1]+1e-5)).sum()
'''

In [ ]:
def portion(mask):
    h,w=mask.shape
    return mask.sum()/(h*w)
#25% okay

In [ ]:
import numpy as np
import pydensecrf.densecrf as dcrf

def dense_crf(img, output_probs):
    h = output_probs.shape[0]
    w = output_probs.shape[1]

    output_probs = np.expand_dims(output_probs, 0)
    output_probs = np.append(1 - output_probs, output_probs, axis=0)
    #output_probs[1] = output_probs[1]+1e-4
    d = dcrf.DenseCRF2D(w, h, 2)
    U = -np.log(output_probs)
    U = U.reshape((2, -1))
    U = np.ascontiguousarray(U)
    #print(U.sum())
    img = np.ascontiguousarray(img)

    d.setUnaryEnergy(U.astype(np.float32))

    d.addPairwiseGaussian(sxy=20, compat=3)
    d.addPairwiseBilateral(sxy=30, srgb=20, rgbim=img, compat=10)

    Q = d.inference(20)
    Q = np.argmax(np.array(Q,dtype=np.float32), axis=0).reshape((h, w))

    return Q

In [ ]:
# Since the submission system does not permit overlapped masks, we have to fix them
def refine_masks(masks,scores):
    areas = np.sum(masks.reshape(-1, masks.shape[-1]), axis=0)
    #mask_index = np.argsort(areas)
    mask_index = np.argsort(scores)[::-1]
    #print(mask_index)
    union_mask = np.zeros(masks.shape[:-1], dtype=bool)
    for m in mask_index:
        masks[:, :, m] = np.logical_and(masks[:, :, m], np.logical_not(union_mask))
        union_mask = np.logical_or(masks[:, :, m], union_mask)
    return masks

In [ ]:
input_dir = "fashion/"
sample_df = pd.read_csv(input_dir + "sample_submission.csv")
'1554' '1764''2184' '984'

In [ ]:
sample_df['ImageId'][0]

In [ ]:
result=model.detect([img])
r = result[0]

In [ ]:
img = np.array(Image.open(DATA_DIR+"/test/"+sample_df['ImageId'][101]))


import cv2

height,width,_=img.shape
if height*width > (1024*1024):
    resize_scale=math.sqrt((height*width)/(1024*1024))
    new_height = int(height/resize_scale)
    new_width = int(width/resize_scale)
    print(height,new_height)
    print(width,new_width)
    img = cv2.resize(img, dsize=(new_width,new_height))



result=model.detect([img])
r = result[0]

In [ ]:
plt.imshow(img)

In [ ]:
r['scores']

In [ ]:
class_names=['BG', 'shirt', 'top', 'sweater', 'cardigan', 'jacket', 'vest', 'pants', 'shorts', 'skirt', 'coat', 'dress', 'jumpsuit', 'cape',
 'glasses', 'hat', 'headband', 'tie', 'glove', 'watch', 'belt', 'leg warmer', 'tights', 'sock', 'shoe', 'bag', 'scarf', 'umbrella', 'hood',
 'collar', 'lapel', 'epaulette', 'sleeve', 'pocket', 'neckline', 'buckle', 'zipper', 'applique', 'bead', 'bow', 'flower', 'fringe',
 'ribbon', 'rivet', 'ruffle', 'sequin', 'tassel']

In [ ]:
visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                            class_names)

In [ ]:
r['scores']

In [ ]:
r['class_ids']

In [ ]:
THRESHOLD_SCORE=0.1

In [ ]:
molded_masks=r['masks'][:,:,r['scores']>THRESHOLD_SCORE]
#raw_masks=r['raw_masks'][:,:,r['scores']>THRESHOLD_SCORE]
molded_masks=refine_masks(molded_masks,r['scores'][r['scores']>THRESHOLD_SCORE])

i=(r['scores']>THRESHOLD_SCORE).sum()

In [ ]:
visualize.display_instances(img, r['rois'][:i], molded_masks, r['class_ids'][:i], 
                            class_names)

In [ ]:
i=3
if i > -1:
    visualize.display_instances(img, np.expand_dims(r['rois'][i],0), np.expand_dims(molded_masks[:,:,i],-1), np.expand_dims(r['class_ids'][i],0), 
                            class_names)

In [ ]:
debug=False
if debug:
    raw_masks = raw_masks*molded_masks
#plt.imshow(raw_masks[:,:,1])

In [ ]:
(raw_masks<0.01).sum()

In [ ]:
bboxs.shape

In [ ]:
bboxs=r['rois']

i=4
if i > -1:
    img_seg=img[bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
    mask_seg=raw_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
    molded_mask=molded_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
    if portion(molded_mask) < 0.2:
        keep += 1
    mask_seg = mask_seg+1e-9
    Q = dense_crf(img_seg,mask_seg)
    molded_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]=Q

    visualize.display_instances(img, np.expand_dims(r['rois'][i],0), np.expand_dims(molded_masks[:,:,i],-1), np.expand_dims(r['class_ids'][i],0), 
                                class_names)


In [ ]:

bboxs=r['rois']
#raw_masks=r['raw_masks']
#molded_masks=r['masks']

keep=[]

for i in range(raw_masks.shape[-1]):
    if r['scores'][i] > THRESHOLD_SCORE:
        img_seg=img[bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
        mask_seg=raw_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
        molded_mask=molded_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
        if portion(molded_mask) < 0.2:
            if portion(molded_mask) < 0.05:
                keep.append(False)
                continue
        keep.append(True)
    
        #mask_seg = mask_seg+1e-9
        #Q = dense_crf(img_seg,mask_seg)
        #if Q.sum() > 0:
        #    if abs(portion(molded_mask)-portion(Q)/portion(molded_mask)) > 0.5:
        #        keep.append(True)
                #print(abs(molded_mask.sum()-Q.sum()/molded_mask.sum()))
        #        continue
        #    else:
        #        molded_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]=Q
        #        keep.append(True)    
        #else:
        #    keep.append(False)
            #keep += 1
        #molded_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]=Q
        #keep += 1


print(molded_masks.shape)
result_scores=r['scores'][r['scores']>THRESHOLD_SCORE][keep]
result_masks=molded_masks[:,:,keep]
result_class_ids=r['class_ids'][r['scores']>THRESHOLD_SCORE][keep]
result_rois=r['rois'][r['scores']>THRESHOLD_SCORE][keep]
result_masks=refine_masks(result_masks,result_scores)
#molded_masks=molded_masks[:,:,:keep]

#class_ids=r['class_ids'][:keep]
import cv2
res = cv2.resize(img, dsize=(512, 512), interpolation=cv2.INTER_CUBIC)
#print(rois.shape,class_ids.shape,molded_masks.shape)
#visualize.display_instances(img, result_rois, result_masks, result_class_ids, 
#                            class_names)

In [ ]:
visualize.display_instances(img, result_rois, result_masks, result_class_ids, 
                            class_names)

In [ ]:
bboxs=r['rois']
#raw_masks=r['raw_masks']
#molded_masks=r['masks']

keep=[]

for i in range(raw_masks.shape[-1]):
    if r['scores'][i] > THRESHOLD_SCORE:
        img_seg=img[bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
        mask_seg=raw_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
        molded_mask=molded_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]

        if portion(molded_mask) < 0.20:
            if portion(molded_mask) < 0.05:
                keep.append(False)
                continue
            print(portion(molded_mask))
            keep.append(True)
            continue
        mask_seg = mask_seg+1e-9
        Q = dense_crf(img_seg,mask_seg)
        if Q.sum() > 0:
            if abs((portion(molded_mask)-portion(Q))/portion(molded_mask)) > 0.3:
                keep.append(True)
                print(i,abs((portion(molded_mask)-portion(Q))/portion(molded_mask)))
                continue
            else:
                print(i,abs((portion(molded_mask)-portion(Q))/portion(molded_mask)))
                molded_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]=Q
                keep.append(True)    
        else:
        #    print(i)
            keep.append(False)
            #keep += 1
        #molded_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]=Q
        #keep += 1


print(molded_masks.shape)
result_scores=r['scores'][r['scores']>THRESHOLD_SCORE][keep]
result_masks=molded_masks[:,:,keep]
result_class_ids=r['class_ids'][r['scores']>THRESHOLD_SCORE][keep]
result_rois=r['rois'][r['scores']>THRESHOLD_SCORE][keep]
result_masks=refine_masks(result_masks,result_scores)
bboxs=r['rois'][r['scores']>THRESHOLD_SCORE][keep]
#molded_masks=molded_masks[:,:,:keep]

keep=[]
for i in range(result_masks.shape[-1]):   
    molded_mask=result_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
    if portion(molded_mask) < 0.05:
        print(i,portion(molded_mask))
        keep.append(False)
        continue
         
    keep.append(True)
print(keep)
result_scores=result_scores[keep]
result_masks=result_masks[:,:,keep]
result_class_ids=result_class_ids[keep]
result_rois=result_rois[keep]
result_masks=refine_masks(result_masks,result_scores)

import cv2
res = cv2.resize(img, dsize=(512, 512), interpolation=cv2.INTER_CUBIC)
#print(rois.shape,class_ids.shape,molded_masks.shape)
#visualize.display_instances(img, result_rois, result_masks, result_class_ids, 
#                            class_names)

In [ ]:
visualize.display_instances(img, result_rois, result_masks, result_class_ids, 
                            class_names)

In [ ]:
import cv2
IMAGE_SIZE=512
res_img = cv2.resize(img, dsize=(IMAGE_SIZE, IMAGE_SIZE))
res_masks=np.zeros((IMAGE_SIZE,IMAGE_SIZE,result_masks.shape[-1]))
for m in range(result_masks.shape[-1]):
    res_masks[:, :, m] = cv2.resize(result_masks[:, :, m].astype('uint8'), (IMAGE_SIZE,IMAGE_SIZE), interpolation=cv2.INTER_NEAREST)
y_scale=IMAGE_SIZE/img.shape[0]
x_scale=IMAGE_SIZE/img.shape[1]
res_rois = (result_rois * [y_scale, x_scale, y_scale, x_scale]).astype(int)

In [ ]:
visualize.display_instances(res_img, res_rois, res_masks, result_class_ids, 
                            class_names)

In [ ]:
import itertools
# Convert data to run-length encoding
def to_rle(bits):
    rle = []
    pos = 0
    for bit, group in itertools.groupby(bits):
        group_list = list(group)
        if bit:
            rle.extend([pos, int(sum(group_list))])
        pos += len(group_list)
    return rle

In [ ]:
THRESHOLD_SCORE=0.1

sub_list = []
missing_count = 0
for i_, row in tqdm(sample_df.iterrows(), total=len(sample_df)):
    #image = resize_image(str(DATA_DIR/'test'/row['ImageId']))
    img = np.array(Image.open(DATA_DIR+"/test/"+row['ImageId']))
    
    height,width,_=img.shape
    if height*width > (1024*1024):
        resize_scale=math.sqrt((height*width)/(1024*1024))
        new_height = int(height/resize_scale)
        new_width = int(width/resize_scale)

        img = cv2.resize(img, dsize=(new_width,new_height))   
    
    
    r = model.detect([img])[0]
    if r['masks'].size > 0 and r['scores'][r['scores']>THRESHOLD_SCORE].size > 0:
        molded_masks=r['masks'][:,:,r['scores']>THRESHOLD_SCORE]
        raw_masks=r['raw_masks'][:,:,r['scores']>THRESHOLD_SCORE]
        molded_masks=refine_masks(molded_masks,r['scores'][r['scores']>THRESHOLD_SCORE])
        bboxs=r['rois']
        
        keep=[]

        for i in range(raw_masks.shape[-1]):
            if r['scores'][i] > THRESHOLD_SCORE:
                img_seg=img[bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
                mask_seg=raw_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
                molded_mask=molded_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
                if portion(molded_mask) < 0.2:
                    if portion(molded_mask) < 0.05:
                        keep.append(False)
                        continue
                    keep.append(True)
                    continue
                mask_seg = mask_seg+1e-9
                Q = dense_crf(img_seg,mask_seg)
                if Q.sum() > 0:
                    if abs(portion(molded_mask)-portion(Q)/portion(molded_mask)) > 0.3:
                        keep.append(True)
                        continue
                    else:
                        molded_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]=Q
                        keep.append(True)     
                else:
                    keep.append(False)
                    #keep += 1
                #molded_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]=Q
                #keep += 1


        #print(molded_masks.shape)
        result_scores=r['scores'][r['scores']>THRESHOLD_SCORE][keep]
        result_masks=molded_masks[:,:,keep]
        result_class_ids=r['class_ids'][r['scores']>THRESHOLD_SCORE][keep]
        #result_rois=r['rois'][r['scores']>0.8][keep]
        result_masks=refine_masks(result_masks,result_scores)
        bboxs=r['rois'][r['scores']>THRESHOLD_SCORE][keep]
        #molded_masks=molded_masks[:,:,:keep]

        keep=[]
        for i in range(result_masks.shape[-1]):   
            molded_mask=result_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
            if portion(molded_mask) < 0.05:
                keep.append(False)
                continue

            keep.append(True)
        result_scores=result_scores[keep]
        result_masks=result_masks[:,:,keep]
        result_class_ids=result_class_ids[keep]
        #result_rois=result_rois[keep]
        result_masks=refine_masks(result_masks,result_scores)        
        
        
    
        IMAGE_SIZE=512
        #res_img = cv2.resize(img, dsize=(IMAGE_SIZE, IMAGE_SIZE))
        res_masks=np.zeros((IMAGE_SIZE,IMAGE_SIZE,result_masks.shape[-1]))
        for m in range(result_masks.shape[-1]):
            res_masks[:, :, m] = cv2.resize(result_masks[:, :, m].astype('uint8'), (IMAGE_SIZE,IMAGE_SIZE), interpolation=cv2.INTER_NEAREST)
        #y_scale=IMAGE_SIZE/img.shape[0]
        #x_scale=IMAGE_SIZE/img.shape[1]
        #res_rois = (result_rois * [y_scale, x_scale, y_scale, x_scale]).astype(int)
        #print(res_masks.shape[-1])
        
        res_masks=refine_masks(res_masks,result_scores)
        
        for m in range(res_masks.shape[-1]):

            mask = res_masks[:, :, m].ravel(order='F')
            rle = to_rle(mask)
            label = result_class_ids[m] - 1
            sub_list.append([row['ImageId'], ' '.join(list(map(str, rle))), label])    
        
    else:
        # The system does not allow missing ids, this is an easy way to fill them 
        sub_list.append([row['ImageId'], '1 1', 23])
        missing_count += 1


In [ ]:
submission_df = pd.DataFrame(sub_list, columns=sample_df.columns.values)
print("Total image results: ", submission_df['ImageId'].nunique())
print("Missing Images: ", missing_count)
submission_df.head()

In [ ]:
submission_df.to_csv("advanced_ML/fashion_submission8.csv",index=False)

In [ ]:
sub_df= pd.read_csv("advanced_ML/fashion_submission_5.csv")

In [ ]:
int(float('4.0'))

In [ ]:
' '.join(list(map(str,[int(float(i)) for i in sub_df.iloc[0].EncodedPixels.split()])))

In [ ]:
sub_df.EncodedPixels.apply(lambda x: len([int(float(i)) for i in str(x).split() if float(i)!=float('nan')])).unique()

In [ ]:
' '.join(list(map(str,[int(float(i)) for i in sub_df.iloc[0].EncodedPixels.split()])))

In [ ]:
for i_, row in tqdm(sub_df.iterrows(), total=len(sample_df)):
    if i_ > 0:
        break
    print(row.EncodedPixels)
    

In [ ]:
THRESHOLD_SCORE=0.1

sub_list = []
missing_count = 0
for i_, row in tqdm(sample_df.iterrows(), total=len(sample_df)):

    
    #image = resize_image(str(DATA_DIR/'test'/row['ImageId']))
    img = np.array(Image.open(DATA_DIR+"/test/"+row['ImageId']))
    
    height,width,_=img.shape
    if height*width > (1024*1024):
        resize_scale=math.sqrt((height*width)/(1024*1024))
        new_height = int(height/resize_scale)
        new_width = int(width/resize_scale)

        img = cv2.resize(img, dsize=(new_width,new_height))   
    
    
    
    r = model.detect([img])[0]
    if r['masks'].size > 0 and r['scores'][r['scores']>THRESHOLD_SCORE].size > 0:
        molded_masks=r['masks'][:,:,r['scores']>THRESHOLD_SCORE]
        raw_masks=r['raw_masks'][:,:,r['scores']>THRESHOLD_SCORE]
        molded_masks=refine_masks(molded_masks,r['scores'][r['scores']>THRESHOLD_SCORE])
        bboxs=r['rois']
        
        keep=[]

        for i in range(raw_masks.shape[-1]):
            if r['scores'][i] > THRESHOLD_SCORE:
                img_seg=img[bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
                mask_seg=raw_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
                molded_mask=molded_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]
                if portion(molded_mask) < 0.2:
                    if portion(molded_mask) < 0.05:
                        keep.append(False)
                        continue
                keep.append(True)
                    
                
                #mask_seg = mask_seg+1e-9
                #Q = dense_crf(img_seg,mask_seg)
                #if Q.sum() > 0:
                #    if abs(portion(molded_mask)-portion(Q)/portion(molded_mask)) > 0.5:
                #        keep.append(True)
                #        continue
                #    else:
                #        molded_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]=Q
                #        keep.append(True)     
                #else:
                #    keep.append(False)
                    #keep += 1
                #molded_masks[:,:,i][bboxs[i][0]:bboxs[i][2],bboxs[i][1]:bboxs[i][3]]=Q
                #keep += 1
        #print(keep)

        #print(molded_masks.shape)
        result_scores=r['scores'][r['scores']>THRESHOLD_SCORE][keep]
        result_masks=molded_masks[:,:,keep]
        result_class_ids=r['class_ids'][r['scores']>THRESHOLD_SCORE][keep]
        #result_rois=r['rois'][r['scores']>0.8][keep]
        result_masks=refine_masks(result_masks,result_scores)        
    
        IMAGE_SIZE=512
        #res_img = cv2.resize(img, dsize=(IMAGE_SIZE, IMAGE_SIZE))
        res_masks=np.zeros((IMAGE_SIZE,IMAGE_SIZE,result_masks.shape[-1]))
        for m in range(result_masks.shape[-1]):
            res_masks[:, :, m] = cv2.resize(result_masks[:, :, m].astype('uint8'), (IMAGE_SIZE,IMAGE_SIZE), interpolation=cv2.INTER_NEAREST)
        #y_scale=IMAGE_SIZE/img.shape[0]
        #x_scale=IMAGE_SIZE/img.shape[1]
        #res_rois = (result_rois * [y_scale, x_scale, y_scale, x_scale]).astype(int)
        #print(res_masks.shape[-1])
        
        res_masks=refine_masks(res_masks,result_scores)
        
        for m in range(res_masks.shape[-1]):
            mask = res_masks[:, :, m].ravel(order='F')
            rle = to_rle(mask)
            label = result_class_ids[m] - 1
            sub_list.append([row['ImageId'], ' '.join(list(map(str, rle))), label])    
        
    else:
        # The system does not allow missing ids, this is an easy way to fill them 
        sub_list.append([row['ImageId'], '1 1', 23])
        missing_count += 1

In [ ]:
submission_df = pd.DataFrame(sub_list, columns=sample_df.columns.values)
print("Total image results: ", submission_df['ImageId'].nunique())
print("Missing Images: ", missing_count)
submission_df.head()

In [ ]:
submission_df.to_csv("advanced_ML/fashion_submission_no_crf_2.csv",index=False)

In [ ]:
prev_df = pd.read_csv("advanced_ML/submission_no_crf_1.csv")

submission_df=prev_df.append(submission_df,ignore_index=True)

In [ ]:

submission_df.to_csv("advanced_ML/fashion_submission.csv")

In [ ]:
# Since the submission system does not permit overlapped masks, we have to fix them
def refine_masks_area(masks, rois):
    areas = np.sum(masks.reshape(-1, masks.shape[-1]), axis=0)
    mask_index = np.argsort(areas)
    union_mask = np.zeros(masks.shape[:-1], dtype=bool)
    for m in mask_index:
        masks[:, :, m] = np.logical_and(masks[:, :, m], np.logical_not(union_mask))
        union_mask = np.logical_or(masks[:, :, m], union_mask)
    for m in range(masks.shape[-1]):
        mask_pos = np.where(masks[:, :, m]==True)
        if np.any(mask_pos):
            y1, x1 = np.min(mask_pos, axis=1)
            y2, x2 = np.max(mask_pos, axis=1)
            rois[m, :] = [y1, x1, y2, x2]
    return masks, rois

In [ ]:
THRESHOLD_SCORE=0.7

sub_list = []
missing_count = 0
for i_, row in tqdm(sample_df.iterrows(), total=len(sample_df)):

    
    #image = resize_image(str(DATA_DIR/'test'/row['ImageId']))
    img = np.array(Image.open(DATA_DIR+"/test/"+row['ImageId']))
    
    height,width,_=img.shape
    if height*width > (1024*1024):
        resize_scale=math.sqrt((height*width)/(1024*1024))
        new_height = int(height/resize_scale)
        new_width = int(width/resize_scale)

        img = cv2.resize(img, dsize=(new_width,new_height))   
    
    
    
    r = model.detect([img])[0]
    if r['masks'].size > 0 and r['scores'][r['scores']>THRESHOLD_SCORE].size > 0:
        molded_masks=r['masks'][:,:,r['scores']>THRESHOLD_SCORE]
        #raw_masks=r['raw_masks'][:,:,r['scores']>THRESHOLD_SCORE]
        molded_masks, _ =refine_masks_area(molded_masks,r['rois'][r['scores']>THRESHOLD_SCORE])
        #bboxs=r['rois']       
    
        IMAGE_SIZE=512
        #res_img = cv2.resize(img, dsize=(IMAGE_SIZE, IMAGE_SIZE))
        res_masks=np.zeros((IMAGE_SIZE,IMAGE_SIZE,result_masks.shape[-1]))
        for m in range(result_masks.shape[-1]):
            res_masks[:, :, m] = cv2.resize(result_masks[:, :, m].astype('uint8'), (IMAGE_SIZE,IMAGE_SIZE), interpolation=cv2.INTER_NEAREST)
        #y_scale=IMAGE_SIZE/img.shape[0]
        #x_scale=IMAGE_SIZE/img.shape[1]
        #res_rois = (result_rois * [y_scale, x_scale, y_scale, x_scale]).astype(int)
        #print(res_masks.shape[-1])
        
        #res_masks=refine_masks(res_masks,result_scores)
        
        for m in range(res_masks.shape[-1]):
            mask = res_masks[:, :, m].ravel(order='F')
            rle = to_rle(mask)
            label = result_class_ids[m] - 1
            sub_list.append([row['ImageId'], ' '.join(list(map(str, rle))), label])    
        
    else:
        # The system does not allow missing ids, this is an easy way to fill them 
        sub_list.append([row['ImageId'], '1 1', 23])
        missing_count += 1

In [ ]:
submission_df = pd.DataFrame(sub_list, columns=sample_df.columns.values)
print("Total image results: ", submission_df['ImageId'].nunique())
print("Missing Images: ", missing_count)
submission_df.head()

In [ ]:
submission_df.to_csv("advanced_ML/fashion_submission_no_crf_4.csv",index=False)